# Now Insurance Loss Runs #
## automatic extraction of relevant information in loss reports from pdf files ##

In [13]:
## load depedencies 
import lossrun
import lossrun_models # ORM models
import matplotlib.pyplot as plt
import re

## Pre-processing ##

In [17]:
# Run this to update the pdf files to process

lossrun.transform_to_images_an_entire_folder(pdfs_folder='./data/pdfs/', images_folder='./data/images/', format='.png', log_file='log_file.txt')

lossrun.transform_to_text_an_entire_folder(images_folder='./data/images/', text_folder= './data/txt/', log_file='log_file.txt')

True

In [18]:
## Select file


# pdf raw file no ext 
#__________________________________________
pdf_file = 'MP Care - loss runs 2018-2019'
#___________________________________________

#__________________________________________
#pdf_file = 'AHC-0000343 loss runs PIO-YES0'
#___________________________________________
#....................................................

#__________________________________________
#pdf_file = '2015-16 PKG loss run CHU-NO'
#___________________________________________


#__________________________________________
#pdf_file = 'Admiral LRs0'
#___________________________________________
#
#___________________________________________
#pdf_file = '2015 09 loss Professional Liab WIL-NO'
#__________________________________________


# topic files
topic_file = './data/configuration/config_topics_beta.ino'

# Pre proc datapre_proc
txt_dict, topics, image, image_c = lossrun.pre_proc(pdf_file, 'data', topic_file)

# vizualizate data
plt.figure(figsize=(23,20))
# Orignial data
plt.imshow(image)
# Orignial data

FileNotFoundError: [Errno 2] No such file or directory: './data/txt/MP Care - loss runs 2018-2019.txt'

## Spatial relation filter ##

In [10]:
spatial_filter = lossrun.spatial_filter(txt_dict, topics)
spatial_filter_topics = len(spatial_filter)

for topic in range(spatial_filter_topics):

    string = ' '.join(spatial_filter[topic])
    string = re.sub('\s+',' ', string)
    print(topics[topic][0] + ' ' + str(topics[topic][3]) + ' ' +str(topics[topic][4]) +' ' + string)
    print(string )
    print('.'*100 + '\n')

policy_num 1163 106 - Majoch, Report Includes Reserve 99 Reserve X-RAYS UP. PLAINTIFF Reserve difficulty filed citing for which patient -— Internal use only BO00319X0AS - Group Claims Summary - Sorted by Report Date within Policy -— Internal use only BO00319X0AS
- Majoch, Report Includes Reserve 99 Reserve X-RAYS UP. PLAINTIFF Reserve difficulty filed citing for which patient -— Internal use only BO00319X0AS - Group Claims Summary - Sorted by Report Date within Policy -— Internal use only BO00319X0AS
....................................................................................................

policy_num 209 339  Claim/ Multi Loss 13990 Policy: Claim/ Multi Loss 16077/ 3158 Policy: Claim/ Multi Loss 44305 PSL 0105240
 Claim/ Multi Loss 13990 Policy: Claim/ Multi Loss 16077/ 3158 Policy: Claim/ Multi Loss 44305 PSL 0105240
....................................................................................................

policy_num 209 772  Claim/ Multi Loss 16077/ 3158 Policy:

### Name entity recognition ##

In [8]:
## LOAD NAME MODEL 
import spacy 
nlp = spacy.load('./data/results/models/')

In [9]:
import re
from configobj import ConfigObj
import string as String
printable = set(String.printable)
# topic rules
ner_rules = ConfigObj('./data/configuration/config_rules_beta.ino')

_temp = []
#
for i in range(len(spatial_filter)):

    string = ' '.join(spatial_filter[:][i])
    string = re.sub('\s+',' ',string)
    
    
    # remove non printalbes elemts
    string = ''.join(filter(lambda x: x in printable, string))
    #print (string)
    doc = nlp(string)

    for ent in doc.ents:
        print('It will be added to data base : ')
        print(ent.text + ' in ' + ent.label_)
        print('\n.................')
        


It will be added to data base : 
KKK KK KKK in ALPHANUM

.................
It will be added to data base : 
KKK KK KKK in ALPHANUM

.................


## DATA BASE INSERTION ##

In [13]:
#Sinlge function for database insertion
Lossrun_models.registerRecord(timeDimDay = 12,
                              timeDimMonth = 9,
                              timeDimYear = 20,
                              policyDimStatus = "Open", 
                              reportGeneratorDimName = _temp[encuentra(_temp,'ORG')][0], 
                              insuredDimName = _temp[encuentra(_temp,'ORG')][0],
                              insurerDimName = _temp[encuentra(_temp,'PERSON')][0], 
                              statusName = "Open", 
                              lossRunReportDimDate = datetime.datetime.now()) 

NameError: name 'Lossrun_models' is not defined

## Contextual analysis

In [ ]:
# Call the model 
model = lossrun.load_context_model()

In [ ]:
model.most_similar('carrier')

In [ ]:
similar = []
try:
    similar.append(model.most_similar_to_given('ORG', txt_dict['text']))
except: 
    similar.append('0')

similar

In [ ]:
# test cell ignore it
###############################


# check topic
_topic = 'Paid'

relates_words_index = []

rate_relation =  0.1 # where grammar  correlation goes to -1 to 1 

for i in txt_dict['text']:

    try:
        relates_words_index.append(model.similarity(_topic, i))
    except:
        relates_words_index.append(0)
#model.similarity()
for index, relate_rate in enumerate(relates_words_index):

    if (relate_rate > .2):
        #print(index)
        print(txt_dict['text'][index])

## Filter loss run report from others ##


In [ ]:
path_txt = './data/txt/'
path_im = './data/images/'
_file = '2015 09 loss Professional Liab WIL-NO'
loss_2 = lossrun.read_dict(path_txt + _file + '.txt')
plt.figure(figsize=(20,20))
image  = cv2.imread(path_im + _file + '.png')
plt.imshow(image)

In [ ]:
only_text = []
txt_y, txt_x = [],[]

for i  in range(len(loss_2['text'])):

    if loss_2['text'][i]!= '':

        only_text.append(loss_2['width'][i] * loss_2['height'][i])
        txt_x.append(loss_2['left'][i])
        txt_y.append(loss_2['top'][i]) 
        
# densidad general del texto 
altura = max(txt_y) - min(txt_y)
base = max(txt_x) - min(txt_x)

# densidad por palabra
x = sum(only_text)

sum(only_text) / (altura * base) 


In [ ]:
import spacy 

In [ ]:
# probar la distribución del text 
# centro de masa del texto
# similitd de texo 


nlp = spacy.load('en_core_web_lg')

loss_1 = lossrun.read_dict(path_txt +  'Email from Ashley to decline due to losses'+ '.txt')
loss_1 = ' '.join(loss_1['text'])

loss_2 = lossrun.read_dict(path_txt + 'AHC-0000343 loss runs PIO-YES' + '.txt')
loss_2 = ' '.join(loss_2['text'])

loss_3 = lossrun.read_dict(path_txt + 'madison medical llc loss runs VGM-NO' + '.txt')
loss_3 = ' '.join(loss_3['text'])


string_1 = nlp(loss_1)
string_2 = nlp(loss_2)
string_3 = nlp(loss_3)


In [ ]:
string_2.similarity(string_3)

In [ ]:
nlp = spacy.load('en_core_web_lg')


In [30]:
sent = 'Insured Division Master/Subsidiary BCO Claim Sub Ltr] Valuation Number G2717131A Policy Term 10/28/2013 - 09/01/2014 Policy Number G2717131A Policy Term 09/01/2014 - 09/01/2015'

entity = nlp('insurer')

for i in sent.split():
    doc = nlp (i)
    print(entity.similarity(doc) , i)

0.6801620074936275 Insured
0.30641449276149085 Division
0.2469806626816359 Master/Subsidiary
0.03149815656929901 BCO
0.4265783115031161 Claim
0.14402429732187544 Sub
-0.05077370131100734 Ltr]
0.36892557986386465 Valuation
0.20842582224512837 Number
0.0 G2717131A
0.4060020539406819 Policy
0.2707394799946922 Term
0.0 10/28/2013
-0.0320739006781649 -
0.0 09/01/2014
0.4060020539406819 Policy
0.20842582224512837 Number
0.0 G2717131A
0.4060020539406819 Policy
0.2707394799946922 Term
0.0 09/01/2014
-0.0320739006781649 -
0.0 09/01/2015
